In [1]:
!pip install transformers pandas torch

In [ ]:
# Importar las librerías necesarias
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Cargar la base de datos
data = pd.read_csv('resultado.csv')  # Cambia esta ruta

# Preprocesamiento: asignar clases múltiples
data['Activity'] = data['Activity'].apply(lambda x: x.split(', '))

# Binarizar las etiquetas
label_binarizer = MultiLabelBinarizer()
data['Activity'] = label_binarizer.fit_transform(data['Activity']).tolist()
class_names = label_binarizer.classes_

# Definir el Dataset
class PeptideDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=50):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        sequence = row['Seqence']
        labels = torch.tensor(row['Activity'], dtype=torch.float)
        encoding = self.tokenizer(sequence, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }
        return item

# Inicializar el tokenizador y el modelo de Hugging Face
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd")
model = AutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert_bfd", num_labels=len(class_names))

# Reemplazar la capa de clasificación con una nueva capa usando Sigmoid
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(model.config.hidden_size, len(class_names)),
    torch.nn.Sigmoid()
)

# Dividir los datos en entrenamiento y validación
train_data = data.sample(frac=0.8, random_state=42)
val_data = data.drop(train_data.index)

# Crear datasets personalizados
train_dataset = PeptideDataset(train_data, tokenizer)
val_dataset = PeptideDataset(val_data, tokenizer)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)

# Configurar el optimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Enviar el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Bucle de entrenamiento
epochs = 3
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = torch.nn.BCELoss()(outputs.logits, labels)
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_train_loss / len(train_dataloader)}")

    # Evaluación
    model.eval()
    total_eval_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = torch.nn.BCELoss()(outputs.logits, labels)
            total_eval_loss += loss.item()

            # Predicciones
            predictions = (outputs.logits > 0.5).float()
            correct += (predictions == labels).all(dim=1).sum().item()
            total += labels.size(0)

    avg_eval_loss = total_eval_loss / len(val_dataloader)
    accuracy = correct / total
    print(f"Validation Loss: {avg_eval_loss}, Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoin